In [1]:
import pandas as pd
import os
from datetime import datetime

# 設定檔案路徑
big_dataset_paths = [
    'C:/Users/as/Downloads/largeds1.csv',
    'C:/Users/as/Downloads/largeds2.csv',
    'C:/Users/as/Downloads/largeds3.csv',
    'C:/Users/as/Downloads/largeds4.csv',
    'C:/Users/as/Downloads/largeds5.csv',
    'C:/Users/as/Downloads/largeds6.csv'
]
small_dataset_path = 'C:/Users/as/Downloads/smallds.csv'
output_dir = 'C:/Users/as/Downloads'

# 確保輸出資料夾存在
os.makedirs(output_dir, exist_ok=True)

# 生成唯一的輸出檔案名稱（使用時間戳）
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_path = os.path.join(output_dir, f'merged_dataset_{timestamp}.csv')

# 載入小資料集
try:
    small_dataset = pd.read_csv(small_dataset_path, encoding='utf-8', dtype={0: str}, low_memory=False)
except UnicodeDecodeError:
    print("小資料集編碼錯誤，嘗試 utf-8-sig")
    small_dataset = pd.read_csv(small_dataset_path, encoding='utf-8-sig', dtype={0: str}, low_memory=False)

# 初始化合併後的大資料集
big_dataset_combined = pd.DataFrame()

# 載入並合併所有大資料集
for big_path in big_dataset_paths:
    try:
        big_dataset = pd.read_csv(big_path, encoding='utf-8', dtype={0: str}, low_memory=False)
    except UnicodeDecodeError:
        print(f"大資料集 {big_path} 編碼錯誤，嘗試 utf-8-sig")
        big_dataset = pd.read_csv(big_path, encoding='utf-8-sig', dtype={0: str}, low_memory=False)
    big_dataset_combined = pd.concat([big_dataset_combined, big_dataset], ignore_index=True)

# 檢查資料集大小和欄位
print(f"合併後大資料集行數：{len(big_dataset_combined)}")
print(f"小資料集行數：{len(small_dataset)}")
print("大資料集欄位：", big_dataset_combined.columns.tolist())
print("小資料集欄位：", small_dataset.columns.tolist())

# TODO: 根據列印的欄位名稱，更新以下變數
company_name_col = '公司名稱'  # 替換為實際的公司名稱欄位（例如 '公司'）
industry_col = '產業類別'  # 替換為實際的產業類別欄位（例如 '行業類別')

# 確保公司名稱為字串格式並去除多餘空格
big_dataset_combined[company_name_col] = big_dataset_combined[company_name_col].astype(str).str.strip()
small_dataset[company_name_col] = small_dataset[company_name_col].astype(str).str.strip()

# 清理名稱：去除常見後綴以提高匹配率
big_dataset_combined[company_name_col] = big_dataset_combined[company_name_col].str.replace(
    r'股份有限公司|有限公司|股份公司|\(股\)|公司', '', regex=True).str.lower()
small_dataset[company_name_col] = small_dataset[company_name_col].str.replace(
    r'股份有限公司|有限公司|股份公司|\(股\)|公司', '', regex=True).str.lower()

# 處理空值或無效名稱
big_dataset_combined = big_dataset_combined[big_dataset_combined[company_name_col].notna() & (big_dataset_combined[company_name_col] != '')]
small_dataset = small_dataset[small_dataset[company_name_col].notna() & (small_dataset[company_name_col] != '')]

# 以公司名稱合併，將大資料集的產業類別補充到小資料集
merged_dataset = small_dataset.merge(
    big_dataset_combined[[company_name_col, industry_col]],
    on=company_name_col,
    how='left'
)

# 儲存結果
merged_dataset.to_csv(output_path, encoding='utf-8-sig', index=False)
print(f"已完成精確比對，小資料集已補充產業類別，儲存為 {output_path}")

# 檢查缺失值
missing_count = merged_dataset[industry_col].isna().sum()
if missing_count > 0:
    print(f"警告：有 {missing_count} 筆資料的產業類別缺失，可能是公司名稱在大資料集中不存在或格式不一致。")
    missing_output_path = os.path.join(output_dir, f'missing_industry_{timestamp}.csv')
    missing_data = merged_dataset[merged_dataset[industry_col].isna()]
    missing_data.to_csv(missing_output_path, encoding='utf-8-sig', index=False)
    print(f"缺失產業類別的資料已儲存為 {missing_output_path}")

合併後大資料集行數：4937032
小資料集行數：5831
大資料集欄位： ['統一編號', '公司名稱', '產業類別']
小資料集欄位： ['項目名稱', '選舉名稱', '申報序號／年度', '交易日期', '收支科目', '公司名稱', '身分證／統一編號', '收入金額', '支出金額', '支出用途', '金錢類', '地址', '聯絡電話', '捐贈方式', '存入專戶日期', '返還/繳庫', '應揭露之支出對象', '支出對象之內部人員姓名', '支出對象之內部人員職稱', '政黨之內部人員姓名', '政黨之內部人員職稱', '關係', '更正註記', '資料更正日期']
已完成精確比對，小資料集已補充產業類別，儲存為 C:/Users/as/Downloads\merged_dataset_20250505_113617.csv
警告：有 2122 筆資料的產業類別缺失，可能是公司名稱在大資料集中不存在或格式不一致。
缺失產業類別的資料已儲存為 C:/Users/as/Downloads\missing_industry_20250505_113617.csv
